In [16]:
import pandas as pd
import folium
import requests

In [17]:
df = pd.read_csv("ficheros/areas_limpio.csv", index_col= 0)
df.head()

,latitude,longitude,town,region,country,price,water,wc,grey_water
0,39.06956,-0.40980,Carcaixent,Comunitat Valenciana,España,Free,yes,yes,yes
1,42.64159,-8.89696,Boiro,Galicia,España,Free,yes,yes,unknown
2,42.90715,0.35921,Arreau,Occitanie,France,Paying,yes,no,yes
3,38.39479,-0.41007,el Campello,Comunitat Valenciana,España,Paying,yes,yes,unknown
4,43.37179,-8.44443,A Coruña,Galicia,España,Free,yes,yes,unknown


In [14]:
map = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=5, control_scale=True)

for index, location_info in df.iterrows():
    if location_info["price"] == "Free":
        folium.Marker([location_info["latitude"], location_info["longitude"]], popup=(location_info["town"], index), icon=folium.Icon(color='blue', prefix='fa',icon='van-shuttle')).add_to(map)
    elif location_info["price"] == "Paying":
        folium.Marker([location_info["latitude"], location_info["longitude"]], popup=(location_info["town"], index), icon=folium.Icon(color='red', prefix='fa',icon='van-shuttle')).add_to(map)

In [15]:
map

In [25]:
def clima(indice):
    
    # hacemos la llamada  a la API
    url = f'https://api.open-meteo.com/v1/forecast?latitude={df.loc[indice,"latitude"]}&longitude={df.loc[indice,"longitude"]}&daily=weathercode&timezone=auto'

    dict_clima = {0: "Clear sky", 
                1: "Mainly clear", 
                2: "Partly cloudy", 
                3: "Overcast", 
                45: "Fog", 
                48: "Depositing rime fog", 
                51: "Light Drizzle", 
                53: "Moderate Drizzle", 
                55: "Dense intensity Drizzle", 
                56: "Light Freezing Drizzle", 
                57: "Dense intensity Freezing Drizzle",
                61: "Slight rain",
                63: "Moderate rain",
                65: "Heavy rain",
                66: "Light Freezing Rain",
                67: "Heavy Freezing Rain",
                71: "Slight Snow fall",
                73: "Moderate Snow fall",
                75: "Heavy Snow fall",
                77: "Snow grains",
                80: "Slight Rain showers",
                81: "Moderate Rain showers",
                82: "Violent Rain showers",
                85: "Slight Snow showers",
                86: "Heavy Snow showers",
                95: "Thunderstorm",
                96: "Thunderstorm with slight hai",
                99: "Thunderstorm with heavy hai"}
            
    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason

    respuesta = response.json()
    dict = respuesta["daily"]
    dia = dict["time"]
    code = dict["weathercode"]
    for num, i in enumerate(code):
        for k,v in dict_clima.items():
            if i == k:
                code[num] = v
    df_3 = pd.DataFrame(code, index= dia, columns= ["Weather"]).T
    return df_3

In [26]:
df1 = clima(522)
df1

,2023-01-29,2023-01-30,2023-01-31,2023-02-01,2023-02-02,2023-02-03,2023-02-04
Weather,Overcast,Fog,Fog,Overcast,Fog,Overcast,Partly cloudy


In [ ]:
class Areas:
    def __init__(self, dataf):

        self.dataf = dataf

    # Creamos un método para crear el mapa

    def crear_mapa(self):
        map = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=5, control_scale=True)

        for index, location_info in df.iterrows():
            if location_info["price"] == "Free":
                folium.Marker([location_info["latitude"], location_info["longitude"]], popup=location_info["town"], icon=folium.Icon(color='blue', prefix='fa',icon='van-shuttle')).add_to(map)
            elif location_info["price"] == "Paying":
                folium.Marker([location_info["latitude"], location_info["longitude"]], popup=location_info["town"], icon=folium.Icon(color='red', prefix='fa',icon='van-shuttle')).add_to(map)
        return map
